# Province Gender Available - Detect Active Users

In [1]:
import gzip
import json
import pandas as pd
import numpy as np
from datetime import datetime

## Read Data from Gzip

In [2]:
with gzip.open("province_gender_available_metadata_added-220614_combined.txt.gz", "rb") as f:
    users = f.readlines()
    
for i in range(len(users)):
    users[i] = json.loads(users[i].decode("utf-8"))

In [3]:
user_count = len(users)
print(user_count)

599943


## Create Users DataFrame

In [4]:
df = pd.DataFrame(users)

In [5]:
df.shape

(599943, 15)

In [6]:
del users

In [7]:
## Count of users with no tweet:

users_with_no_tweet_count = 0

for tweet_list in df["tweets"]:
    if tweet_list == []:
        users_with_no_tweet_count += 1
        
print(f"Count of users with no tweet: {users_with_no_tweet_count}")

Count of users with no tweet: 147632


## Detect Active Users

### 1- Mean of Tweet Years

In [8]:
def detect_active_by_mean(tweets, year, tweet_types):
    tweets = [tweet for tweet in tweets if tweet["type"] in tweet_types]
    if not tweets:
        return "passive"
    else:
        mean_tweet_years = np.mean([int(tweet["twt_date"][:2]) for tweet in tweets])
        if mean_tweet_years >= year:
            return "active"
        else:
            return "passive"

In [9]:
active_by_mean = pd.DataFrame(columns=["year", "tweet_types", "active", "passive", "active_percentage"])

for tweet_type_list in [["original", "retweet", "reply", "quote", "fav"],
                        ["original", "reply", "quote"],
                        ["original"]]:
    for year in range(6, 23):
        if year < 10:
            print(f"Year: 200{year} | By Mean Year | Tweet Types: {tweet_type_list}", end="...")
        else:
            print(f"Year: 20{year} | By Mean Year | Tweet Types: {tweet_type_list}", end="...")
        
        user_value_counts = pd.DataFrame(df["tweets"].apply(lambda x: detect_active_by_mean(x, year, tweet_type_list)).value_counts()).rename({"tweets":"users"}, axis=1).reset_index()
        active_count = user_value_counts[user_value_counts["index"] == "active"]["users"].values[0]
        passive_count = user_value_counts[user_value_counts["index"] == "passive"]["users"].values[0]
        active_by_mean = active_by_mean.append(pd.DataFrame({"year": year, "tweet_types": [tweet_type_list], "active": active_count, "passive": passive_count, "active_percentage": active_count/user_count*100}), ignore_index=True).reset_index(drop=True)
        
        print("completed!")

Year: 2006 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2007 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2008 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2009 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2010 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2011 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2012 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2013 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2014 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2015 | By Mean Year | Tweet Types: ['original', 'retweet', 'reply',

In [10]:
active_by_mean["year"] = active_by_mean["year"].apply(lambda x: f"200{x}" if x < 10 else f"20{x}")
active_by_mean["technique"] = "mean"
active_by_mean = active_by_mean[["year", "tweet_types", "technique", "active", "passive", "active_percentage"]]

In [11]:
active_by_mean.to_csv("active_users_by_mean.csv", index=False)

In [12]:
display(active_by_mean)

,year,tweet_types,technique,active,passive,active_percentage
0,2006,"[original, retweet, reply, quote, fav]",mean,452311,147632,75.392329
1,2007,"[original, retweet, reply, quote, fav]",mean,452311,147632,75.392329
2,2008,"[original, retweet, reply, quote, fav]",mean,452311,147632,75.392329
3,2009,"[original, retweet, reply, quote, fav]",mean,452309,147634,75.391996
4,2010,"[original, retweet, reply, quote, fav]",mean,452286,147657,75.388162
5,2011,"[original, retweet, reply, quote, fav]",mean,451809,148134,75.308654
6,2012,"[original, retweet, reply, quote, fav]",mean,446985,152958,74.504578
7,2013,"[original, retweet, reply, quote, fav]",mean,436939,163004,72.830086
8,2014,"[original, retweet, reply, quote, fav]",mean,413981,185962,69.003389
9,2015,"[original, retweet, reply, quote, fav]",mean,385422,214521,64.243103


### 2- At Least one Tweet from a Year

In [13]:
def detect_active_by_at_least_one(tweets, year, tweet_types):
    tweets = [tweet for tweet in tweets if tweet["type"] in tweet_types]
    if not tweets:
        return "passive"
    else:
        tweet_years = [int(tweet["twt_date"][:2]) for tweet in tweets]
        if any([tweet_year >= year for tweet_year in tweet_years]):
            return "active"
        else:
            return "passive"

In [14]:
active_by_at_least_one = pd.DataFrame(columns=["year", "tweet_types", "active", "passive", "active_percentage"])

for tweet_type_list in [["original", "retweet", "reply", "quote", "fav"],
                        ["original", "reply", "quote"],
                        ["original"]]:
    for year in range(6, 23):
        if year < 10:
            print(f"Year: 200{year} | By At Least One | Tweet Types: {tweet_type_list}", end="...")
        else:
            print(f"Year: 20{year} | By At Least One | Tweet Types: {tweet_type_list}", end="...")
        
        user_value_counts = pd.DataFrame(df["tweets"].apply(lambda x: detect_active_by_at_least_one(x, year, tweet_type_list)).value_counts()).rename({"tweets":"users"}, axis=1).reset_index()
        active_count = user_value_counts[user_value_counts["index"] == "active"]["users"].values[0]
        passive_count = user_value_counts[user_value_counts["index"] == "passive"]["users"].values[0]
        active_by_at_least_one = active_by_at_least_one.append(pd.DataFrame({"year": year, "tweet_types": [tweet_type_list], "active": active_count, "passive": passive_count, "active_percentage": active_count/user_count*100}), ignore_index=True).reset_index(drop=True)
        
        print("completed!")

Year: 2006 | By At Least One | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2007 | By At Least One | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2008 | By At Least One | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2009 | By At Least One | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2010 | By At Least One | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2011 | By At Least One | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2012 | By At Least One | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2013 | By At Least One | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2014 | By At Least One | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Year: 2015 | By At Least One | Tweet Types: ['

In [15]:
active_by_at_least_one["year"] = active_by_at_least_one["year"].apply(lambda x: f"200{x}" if x < 10 else f"20{x}")
active_by_at_least_one["technique"] = "at_least_one"
active_by_at_least_one = active_by_at_least_one[["year", "tweet_types", "technique", "active", "passive", "active_percentage"]]

In [16]:
active_by_at_least_one.to_csv("active_users_by_at_least_one.csv", index=False)

In [17]:
display(active_by_at_least_one)

,year,tweet_types,technique,active,passive,active_percentage
0,2006,"[original, retweet, reply, quote, fav]",at_least_one,452311,147632,75.392329
1,2007,"[original, retweet, reply, quote, fav]",at_least_one,452311,147632,75.392329
2,2008,"[original, retweet, reply, quote, fav]",at_least_one,452311,147632,75.392329
3,2009,"[original, retweet, reply, quote, fav]",at_least_one,452310,147633,75.392162
4,2010,"[original, retweet, reply, quote, fav]",at_least_one,452296,147647,75.389829
5,2011,"[original, retweet, reply, quote, fav]",at_least_one,451971,147972,75.335657
6,2012,"[original, retweet, reply, quote, fav]",at_least_one,447964,151979,74.667760
7,2013,"[original, retweet, reply, quote, fav]",at_least_one,440280,159663,73.386972
8,2014,"[original, retweet, reply, quote, fav]",at_least_one,423600,176343,70.606708
9,2015,"[original, retweet, reply, quote, fav]",at_least_one,401252,198691,66.881687


### 3- Last N Years

In [18]:
def convert_date(date):
    return date.split("-")[0][2:] + date.split("-")[1] + date.split("-")[2]

In [19]:
def detect_active_by_last_n_years(tweets, n, tweet_types):
    # Filter by tweets
    tweets = [tweet for tweet in tweets if tweet["type"] in tweet_types]
    # Determine threshold by last n years
    now = convert_date(str(datetime.now()).split()[0])
    n = n
    threshold = int(str(int(now[:2]) - n) + now[2:])
    
    if not tweets:
        return "passive"
    else:
        tweet_dates = [int(tweet["twt_date"]) for tweet in tweets]
        if any([tweet_date >= threshold for tweet_date in tweet_dates]):
            return "active"
        else:
            return "passive"

In [20]:
active_by_last_n_years = pd.DataFrame(columns=["last_n_years", "threshold", "tweet_types", "active", "passive", "active_percentage"])
now = convert_date(str(datetime.now()).split()[0])

for tweet_type_list in [["original", "retweet", "reply", "quote", "fav"],
                        ["original", "reply", "quote"],
                        ["original"]]:
    for year in range(1, 17):
        print(f"Last {year} year(s) | By Last N Year | Tweet Types: {tweet_type_list}", end="...")
        
        user_value_counts = pd.DataFrame(df["tweets"].apply(lambda x: detect_active_by_last_n_years(x, year, tweet_type_list)).value_counts()).rename({"tweets":"users"}, axis=1).reset_index()
        active_count = user_value_counts[user_value_counts["index"] == "active"]["users"].values[0]
        passive_count = user_value_counts[user_value_counts["index"] == "passive"]["users"].values[0]
        threshold = str(int(now[:2]) - year) + now[2:]
        if len(threshold) <= 5:
            threshold = "0" + threshold
        
        active_by_last_n_years = active_by_last_n_years.append(pd.DataFrame({"last_n_years": year,
                                                                             "threshold": str(pd.to_datetime(threshold, yearfirst=True)).split()[0],
                                                                             "tweet_types": [tweet_type_list],
                                                                             "active": active_count,
                                                                             "passive": passive_count,
                                                                             "active_percentage": active_count/user_count*100}),
                                                               ignore_index=True).reset_index(drop=True)
        
        print("completed!")

Last 1 year(s) | By Last N Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Last 2 year(s) | By Last N Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Last 3 year(s) | By Last N Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Last 4 year(s) | By Last N Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Last 5 year(s) | By Last N Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Last 6 year(s) | By Last N Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Last 7 year(s) | By Last N Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Last 8 year(s) | By Last N Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Last 9 year(s) | By Last N Year | Tweet Types: ['original', 'retweet', 'reply', 'quote', 'fav']...completed!
Last 10 year(s) | B

In [21]:
display(active_by_last_n_years)

,last_n_years,threshold,tweet_types,active,passive,active_percentage
0,1,2021-06-24,"[original, retweet, reply, quote, fav]",148511,451432,24.754185
1,2,2020-06-24,"[original, retweet, reply, quote, fav]",197604,402339,32.937129
2,3,2019-06-24,"[original, retweet, reply, quote, fav]",245162,354781,40.864215
3,4,2018-06-24,"[original, retweet, reply, quote, fav]",286224,313719,47.708532
4,5,2017-06-24,"[original, retweet, reply, quote, fav]",328490,271453,54.753535
5,6,2016-06-24,"[original, retweet, reply, quote, fav]",364831,235112,60.810944
6,7,2015-06-24,"[original, retweet, reply, quote, fav]",389275,210668,64.885331
7,8,2014-06-24,"[original, retweet, reply, quote, fav]",413215,186728,68.875710
8,9,2013-06-24,"[original, retweet, reply, quote, fav]",434062,165881,72.350540
9,10,2012-06-24,"[original, retweet, reply, quote, fav]",444997,154946,74.173213


In [22]:
active_by_last_n_years["technique"] = "last_n_years"
active_by_last_n_years = active_by_last_n_years[["last_n_years", "tweet_types", "technique", "active", "passive", "active_percentage"]]

In [23]:
active_by_last_n_years.to_csv("active_users_by_last_n_years.csv", index=False)